# nbforms Demo Notebook

The nbforms package allows you to have interactive questions in Jupyter Notebooks that is designed to allow immediate usage of collected data.

### Setup

Before using nbforms, you must provide a config file. The default location of the config file is `./nbforms_config.py`; the cell below loads the config file for this notebook for your perusal.

In [ ]:
with open("./nbforms_config.json") as f:
    print(f.read())

nbforms requires a server to collect responses. The cells below clone the nbforms server repo, start a local development server that you can use for this notebook, and perform some setup tasks in the server instance's database.

In [ ]:
%%bash --bg --out sh_out --err sh_err
if ! [ -d nbforms-server ]; then git clone https://github.com/chrispyles/nbforms-server; fi
cd nbforms-server
CREATE="false"
if ! [ -d .venv ]; then python -m venv .venv; CREATE="true"; fi
source .venv/bin/activate
if [ $CREATE = "true" ]; then pip install -qr requirements.txt; fi
flask --app nbforms_server run --port 8000

In [ ]:
# Only run this cell once. It will show you the output from the bash script in the cell above, which
# is running in a background process.

import asyncio
from IPython.display import Pretty

async def reader(stream, d):
    text = ""
    while not stream.at_eof():
        text += (await stream.readline()).decode()
        d.update(Pretty(text))

d1, d2 = display(display_id="stream-1"), display(display_id="stream-2")
asyncio.create_task(reader(sh_out, d1))
asyncio.create_task(reader(sh_err, d2));

In [ ]:
%%bash
cd nbforms-server
source .venv/bin/activate
python -m nbforms_server attendance open attendance-open --create

### Usage

To use nbforms, create a `Form` instance. This will load the config file and ask the use to authenticate with the server, generating an API token for this notebook session. Since the server has no users, a user will be created with whatever credentials you enter.

In [ ]:
import nbforms
form = nbforms.Form()

To ask a user to respond to a question, call `Form.ask`  with the question _identifier(s)_ as its argument(s). The widget generated will have a "Submit" button which will send a request to your nbforms server that will record the user's response. `Form.ask` can accept multiple arguments and will display a widget for each identifier you pass it. If you pass no arguments, it will display all the questions.

nbforms allows multiple choice (with one or many selections), text, and paragraph responses. An example of each is given below.

In [ ]:
form.ask()

#### Extracting Data from the Server

When retrieving the data from the server, nbforms allows you to collect the data into a datascience `Table` or a pandas `DataFrame`. To get the data from the server, use `Form.to_table` or `Form.to_df` and provide the question identifiers you would like to select. The optional `user_hashes` tells the server whether or not to include an randomly generated hash of each username in the CSV. *In order to protect user data, you cannot retrieve the usernames of users in the CSV.*

In [ ]:
form.to_table()

In [ ]:
form.to_df("q1", "q4", "q5", user_hashes=True)

**Important:** The server route that these methods use to retrieve the data requires no authentication. This means that anyone with the server URL will be able to download any of the data in the responses provided by users. Be very careful about what kind of data you ask users to input to the server; it should not be used to store things like [PII](https://en.wikipedia.org/wiki/Personal_data).

### Taking Attendance

nbforms can be used to take attendance in classes, allowing you to open and close a notebook for attendance tracking. This is done by including an `attendance` key in the config file set to `true`. This will then allow students to run `Form.take_attendance` which will log their attendance on the server.

In [ ]:
with open("attendance_open_config.json") as f:
    print("OPEN ATTENDANCE:")
    print(f.read()[:100])
    
print("\n")

with open("attendance_closed_config.json") as f:
    print("CLOSED ATTENDANCE:")
    print(f.read()[:100])

We instantiate them below and take attendance on each one.

In [ ]:
always_open = nbforms.Form("attendance_open_config.json")
always_open.take_attendance()

In [ ]:
always_closed = nbforms.Form("attendance_closed_config.json")
always_closed.take_attendance()

Attendance is always logged, so even if an attendance form is closed, a student will not know and this will be included in the attendance report.

Using the server's CLI, you can run reports for users, notebooks, responses, and attendance.

In [ ]:
%%bash
cd nbforms-server
source .venv/bin/activate
echo "Responses report for notebook 1:" && echo
python -m nbforms_server reports responses 1

In [ ]:
%%bash
cd nbforms-server
source .venv/bin/activate
echo "Attendance report for notebook attendance-open:" && echo
python -m nbforms_server reports attendance attendance-open

In [ ]:
%%bash
cd nbforms-server
source .venv/bin/activate
echo "Attendance report for notebook attendance-closed:" && echo
python -m nbforms_server reports attendance attendance-closed